<a href="https://colab.research.google.com/github/bobir01/cau_ml_homeworks/blob/cau-kag%3Bgle/kaggle_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
# prompt: import everything that needs for ml, data processing

import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import LabelEncoder
import sklearn
import xgboost as xgb

import re

%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls




# ML Algorithms used
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.svm import SVC
from imblearn.over_sampling import KMeansSMOTE
from sklearn.model_selection import RandomizedSearchCV

!wget "https://drive.google.com/uc?export=download&id=10nXgj4o0MpKafrL9ZbaMTwLoqb1jgctg" -O utils.py
!wget "https://drive.google.com/uc?export=download&id=10nXgj4o0MpKafrL9ZbaMTwLoqb1jgctg&confirm=t&uuid=53c62931-a26d-4e76-b6a7-b79abc3c0749&at=AB6BwCD7w2mZ-0BCoEcElMkDOLDb:1702748115074" -O utils.py

import utils


--2023-12-16 18:55:29--  https://drive.google.com/uc?export=download&id=10nXgj4o0MpKafrL9ZbaMTwLoqb1jgctg
Resolving drive.google.com (drive.google.com)... 142.250.107.100, 142.250.107.138, 142.250.107.101, ...
Connecting to drive.google.com (drive.google.com)|142.250.107.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘utils.py’

utils.py                [ <=>                ]   2.22K  --.-KB/s    in 0s      

2023-12-16 18:55:29 (17.4 MB/s) - ‘utils.py’ saved [2271]

--2023-12-16 18:55:29--  https://drive.google.com/uc?export=download&id=10nXgj4o0MpKafrL9ZbaMTwLoqb1jgctg&confirm=t&uuid=53c62931-a26d-4e76-b6a7-b79abc3c0749&at=AB6BwCD7w2mZ-0BCoEcElMkDOLDb:1702748115074
Resolving drive.google.com (drive.google.com)... 142.250.107.100, 142.250.107.138, 142.250.107.101, ...
Connecting to drive.google.com (drive.google.com)|142.250.107.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https:

In [4]:

test_df = utils.load_dataset(utils.DatasetType.Test)

In [5]:
# df = pd.read_csv(Path('/content/drive/MyDrive/Colab Notebooks/datasets/kaggle_train_data_public.csv'))


train_df = utils.load_dataset(utils.DatasetType.Train)


In [6]:
#@title data pre-processing

train_df.drop(['CustomerID', 'Count', 'Lat Long', 'Churn Reason', 'Country', 'Country',
               'State', 'Zip Code', 'Latitude', 'Longitude',
         'City', 'Churn Score' , 'CLTV'
         ], axis=1, inplace=True)
categorocal_columns = train_df.select_dtypes(['object']).columns.to_list()




for col in categorocal_columns:
    le = LabelEncoder()
    train_df[col] = le.fit_transform( train_df[col] )
    print(col)

Gender
Senior Citizen
Partner
Dependents
Phone Service
Multiple Lines
Internet Service
Online Security
Online Backup
Device Protection
Tech Support
Streaming TV
Streaming Movies
Contract
Paperless Billing
Payment Method


In [ ]:
test_df.drop(['CustomerID', 'Count', 'Lat Long', 'Churn Reason', 'Country', 'Country',
               'State', 'Zip Code', 'Latitude', 'Longitude',
         'City', 'Churn Score' , 'CLTV'
         ], axis=1, inplace=True, errors='ignore')
categorocal_columns = test_df.select_dtypes(['object']).columns.to_list()
for col in categorocal_columns:
    le = LabelEncoder()
    test_df[col] = le.fit_transform( test_df[col] )
    print(col)

In [ ]:
train_df.describe()

In [8]:
kmeans_smote = SMOTENC(random_state=42, categorical_features=[0,1])


X_resampled, y_resampled = kmeans_smote.fit_resample(train_df.drop('Churn Value', axis=1), train_df['Churn Value'])


In [9]:
X_resampled[['Tenure Months', 'Monthly Charges']] = StandardScaler().fit_transform(X_resampled[['Tenure Months', 'Monthly Charges']])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled.ravel(),  test_size=0.2, random_state=42)

In [104]:
X_train, y_train , X_test = X_resampled.values,   y_resampled, test_df.values

In [108]:
X_train.shape

(5798, 18)

In [109]:
# Model Class to be used for different ML algorithms
class ClassifierModel(object):
    def __init__(self, clf, params=None):
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def fit(self,x,y):
        return self.clf.fit(x,y)

    def feature_importances(self,x,y):
        return self.clf.fit(x,y).feature_importances_

    def predict(self, x):
        return self.clf.predict(x)

    def predict_proba(self, x):
        return self.clf.predict_proba(x)

In [113]:
def trainModel(model, x_train, y_train, x_test, n_folds, seed):
    cv = KFold(n_splits= n_folds)

    # rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    Fit the random search model

    # scores = cross_val_score(model.clf, x_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
    # y_pred = cross_val_predict(model.clf, x_test, cv=cv, n_jobs=-1, method='predict_proba')
    model.fit(x_train, y_train)
    y_pred = model.predict_proba(x_test)


    return 0, y_pred[:,1]

In [153]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.



RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [164]:
rf_random.best_params_

{'n_estimators': 1400,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 100,
 'bootstrap': True}

In [157]:
rfc_train_pred = rf_random.predict_proba(X_test)[:,1]


In [146]:
#@title Random Forest parameters
rf_params = {
    'n_estimators': 400,
    'max_depth': 5,
    'min_samples_leaf': 3,
    'max_features' : 'sqrt',
    'random_state': 42
}
rfc_model = ClassifierModel(clf=RandomForestClassifier, params=rf_params)
rfc_scores, rfc_train_pred = trainModel(rfc_model, X_train, y_train, X_test, 5, 0) # Random Forest
rfc_scores

0

In [116]:
rfc_train_pred.shape

(986,)

In [ ]:
X_test

In [50]:
# Getting features importance
rfc_features = rfc_model.feature_importances(X_train,y_train)
rfc_features

array([0.00251663, 0.00097988, 0.03298881, 0.12277623, 0.10694145,
       0.00158873, 0.00357559, 0.03068252, 0.13767904, 0.02287563,
       0.02512068, 0.15110462, 0.00572965, 0.00463313, 0.27883907,
       0.00173616, 0.01274797, 0.05748423])

In [117]:
#@title Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':400,
    'max_depth': 5,
    'min_samples_leaf': 2,
}
etc_model = ClassifierModel(clf=ExtraTreesClassifier, params=et_params)
etc_scores, etc_train_pred = trainModel(etc_model,X_train, y_train, X_test, 5, 0) # Random Forest
etc_scores

0

In [52]:
# Getting features importance
etc_features = etc_model.feature_importances(X_train, y_train)
etc_features

array([0.00206253, 0.00077538, 0.06779841, 0.15312706, 0.0598944 ,
       0.0017379 , 0.00257368, 0.05270563, 0.10702454, 0.02975349,
       0.03648153, 0.13932384, 0.00623492, 0.00580931, 0.29548586,
       0.00218405, 0.00955916, 0.02746828])

In [118]:
#@title  AdaBoost parameters
ada_params = {
    'n_estimators': 400,
    'learning_rate' : 0.65
}
ada_model = ClassifierModel(clf=AdaBoostClassifier, params=ada_params)
ada_scores, ada_train_pred = trainModel(ada_model,X_train, y_train, X_test, 5, 0) # Random Forest
ada_scores

0

In [54]:
# Getting features importance
ada_features = ada_model.feature_importances(X_train, y_train)
ada_features

array([0.    , 0.005 , 0.    , 0.005 , 0.08  , 0.035 , 0.045 , 0.06  ,
       0.0925, 0.07  , 0.12  , 0.0625, 0.04  , 0.03  , 0.0175, 0.0025,
       0.0125, 0.3225])

In [119]:
#@title Gradient Boosting parameters
gb_params = {
    'n_estimators': 400,
    'max_depth': 6,
}
gbc_model = ClassifierModel(clf=GradientBoostingClassifier, params=gb_params)
gbc_scores, gbc_train_pred = trainModel(gbc_model,X_train, y_train, X_test, 5, 0) # Random Forest
gbc_scores

0

array([[9.98314483e-01, 1.68551709e-03],
       [9.99287095e-01, 7.12904601e-04],
       [2.17502415e-03, 9.97824976e-01],
       ...,
       [9.33589376e-03, 9.90664106e-01],
       [7.75310231e-02, 9.22468977e-01],
       [3.41987064e-02, 9.65801294e-01]])

In [56]:
# Getting features importance
gbc_features = gbc_model.feature_importances(X_train,y_train)
gbc_features

array([0.01870896, 0.011668  , 0.01298071, 0.08820383, 0.1249064 ,
       0.01030333, 0.01500047, 0.0040668 , 0.03395331, 0.01385793,
       0.01562739, 0.03187381, 0.00858082, 0.01029316, 0.3161759 ,
       0.01198535, 0.03356861, 0.23824522])

In [120]:
#@title  Support Vector Classifier parameters
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025,
    'probability': True}
svc_model = ClassifierModel(clf=SVC, params=svc_params)
svc_scores, svc_train_pred = trainModel(svc_model,X_train, y_train, X_test, 5, 0) # Random Forest
svc_scores

0

In [61]:
svc_train_pred

array([[0.98283518, 0.01716482],
       [0.97053799, 0.02946201],
       [0.14249825, 0.85750175],
       ...,
       [0.19619711, 0.80380289],
       [0.63021806, 0.36978194],
       [0.17836524, 0.82163476]])

In [59]:
# Create a dataframe with features
feature_dataframe = pd.DataFrame( {'features': X_resampled.columns.values,
    'Random Forest': rfc_features,
    'Extra Trees': etc_features,
    'AdaBoost': ada_features,
    'Gradient': gbc_features
    })

# Create the new column containing the average of values
feature_dataframe['mean'] = feature_dataframe.mean(axis= 1) # axis = 1 computes the mean row-wise

<ipython-input-59-0bfca7670232>:10: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [ ]:
#@title feature importance plot
feature_dataframe.set_index('features').plot.bar(legend=True, title='Feature importance by different models', figsize=(12,8))

In [63]:
rfc_scores

array([0.90085464, 0.88455124, 0.90697815, 0.88485765, 0.86976153])

In [73]:
acc_pred_train = pd.DataFrame ({'RandomForest': rfc_scores.ravel(),
     'ExtraTrees': etc_scores.ravel(),
     'AdaBoost': ada_scores.ravel(),
      'GradientBoost': gbc_scores.ravel(),
'SVC': svc_scores.ravel()
    })
acc_pred_train

,RandomForest,ExtraTrees,AdaBoost,GradientBoost,SVC
0,0.900855,0.894352,0.917866,0.923456,0.889103
1,0.884551,0.882356,0.906710,0.921323,0.878008
2,0.906978,0.899396,0.930230,0.932724,0.897838
3,0.884858,0.881354,0.910041,0.922441,0.872699
4,0.869762,0.863179,0.891961,0.906597,0.857276


In [158]:
# prompt: make one dataframe using all ensamble results

acc_pred = pd.DataFrame ({'RandomForest': rfc_train_pred.ravel(),
    #  'ExtraTrees': etc_train_pred.ravel(),
    #  'AdaBoost': ada_train_pred.ravel(),
    #   'GradientBoost': gbc_train_pred.ravel(),
# 'SVC': svc_train_pred.ravel()
    })

acc_pred


,RandomForest
0,0.683571
1,0.704286
2,0.610714
3,0.235714
4,0.106429
...,...
981,0.373571
982,0.426429
983,0.483571
984,0.339286


In [142]:
acc_pred.describe()

,RandomForest
count,986.000000
mean,0.406332
std,0.228855
min,0.051531
25%,0.218323
50%,0.366871
75%,0.658083
max,0.780467


In [160]:
acc_pred['mean']  = acc_pred.mean(axis=1)

In [161]:
acc_pred[['mean']]

,mean
0,0.683571
1,0.704286
2,0.610714
3,0.235714
4,0.106429
...,...
981,0.373571
982,0.426429
983,0.483571
984,0.339286


In [163]:
pd.DataFrame(data={'Churn Values': acc_pred['mean'].values}, index=pd.Index(acc_pred.index.values, name='ID'), ).to_csv('all_ensamble.csv')

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(7,7))
plt.title('Pearson correlation of features', y=1.05, size=15)
sns.heatmap(acc_pred_train.astype(float).corr(), annot=True)

In [ ]:
y = feature_dataframe['mean'].values
x = feature_dataframe['features'].values
data = [go.Bar(
            x= x,
             y= y,
            width = 0.5,
            marker=dict(
               color = feature_dataframe['mean'].values,
            colorscale='Portland',
            showscale=True,
            reversescale = False
            ),
            opacity=0.6
        )]

layout= go.Layout(
    autosize= True,
    title= 'Barplots of Mean Feature Importance',
    hovermode= 'closest',
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='bar-direct-labels')

In [25]:
x_train = np.column_stack(( etc_train_pred, rfc_train_pred, ada_train_pred, gbc_train_pred, svc_train_pred))

In [31]:
etc_train_pred

array([0, 0, 1, ..., 1, 0, 1])

In [30]:
x_train.sum(axis=0)

array([2509, 2565, 2500, 2388, 2589])

In [27]:
acc_pred_train

,RandomForest,ExtraTrees,AdaBoost,GradientBoost
0,0.815733,0.804957,0.839440,0.857759
1,0.798491,0.802802,0.813578,0.844828
2,0.839440,0.826509,0.850216,0.858836
3,0.805825,0.797195,0.830636,0.845739
4,0.806904,0.795038,0.826321,0.839266


In [ ]:
x_train

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1],
       ...,
       [1, 1, 1, 1, 1],
       [0, 1, 1, 1, 0],
       [1, 1, 1, 1, 1]])

In [33]:
def trainStackModel(x_train, y_train, x_test, n_folds, seed):
    cv = KFold(n_splits= n_folds,)
    gbm = xgb.XGBClassifier(
     n_estimators= 2000,
     max_depth= 4,
     min_child_weight= 2,
     gamma=0.9,
     subsample=0.8,
     colsample_bytree=0.8,
     objective= 'binary:logistic',
     scale_pos_weight=1).fit(x_train, y_train)

    scores = cross_val_score(gbm, x_train, y_train, scoring='accuracy', cv=cv)
    return scores, gbm

In [34]:
stackModel_scores, clf = trainStackModel(x_train, y_train, X_test, 5, 0)

In [ ]:
test_df

In [ ]:
clf.predict(test_df.values)

In [36]:
acc_pred_train['stackingModel'] = stackModel_scores
acc_pred_train.mean(axis=0)

RandomForest     0.813279
ExtraTrees       0.805300
AdaBoost         0.832038
GradientBoost    0.849286
stackingModel    0.848207
dtype: float64

,RandomForest,ExtraTrees,AdaBoost,GradientBoost,stackingModel
0,0.815733,0.804957,0.839440,0.857759,0.860991
1,0.798491,0.802802,0.813578,0.844828,0.835129
2,0.839440,0.826509,0.850216,0.858836,0.866379
3,0.805825,0.797195,0.830636,0.845739,0.836030
4,0.806904,0.795038,0.826321,0.839266,0.842503


In [ ]:
#@title some pics of corr after smot + standart scaler
# colormap = plt.cm.RdBu
# plt.figure(figsize=(16,16))
# plt.title('Pearson correlation of features')
# sns.heatmap(X_train.astype(float).corr(), annot=True)

In [ ]:
#@title some pics of corr without standart scaler
plt.figure(figsize=(16, 16))

sns.heatmap(train_df.corr(), annot=True)
plt.show()

In [ ]:
# graph = sns.FacetGrid(train_df, col='Churn Value')
# graph.map(sns.histplot, "Tenure Months",kde=True, stat="count")

# graph = sns.FacetGrid(train_df, col='Churn Value')
# graph.map(sns.histplot, "Monthly Charges",kde=True, stat="count")